# Projects markdown generator for academicpages

Takes a TSV of projects with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [74]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, slides_url, video_url, excerpt, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [75]:
!cat projects.json

[
    {
        "title": "Extending Whisper, OpenAI's Speech-to-Text Model",
        "contributors": "Abhishek Divekar, Yosub Jung, Roshni Tayal",
        "type": "Academic",
        "project_date": "2022-12-05", 
        "url_slug": "ut-mscs-case-studies-in-ml-project", 
        "teaser_url": "whisper-wer-noisy-vs-quiet.png",
        "report_url": "https://adivekar-utexas.github.io/files/UTCS-ML-Case-Studies-Extending-Whisper-OpenAIs-Speech-to-Text-Model.pdf",
        "code_url": "",
        "excerpt": "We study the performance of OpenAI's Whisper model, the state-of-the-art Speech-to-text model, in noisy urban environments. To do so, we create a dataset consisting of 134 minutes of us reading out loud in both quiet and noisy urban environments (subway, street and cafe) and manually annotating the recordings at 30 second intervals. Using a powerful multi-GPU AWS cluster and distributed computing framework Ray, we find that Whisper performs significantly worse on speeches recorded in n

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [76]:
projects = pd.read_json("projects.json")
projects

,title,contributors,type,project_date,url_slug,teaser_url,report_url,code_url,excerpt,my_contribution
0,"Extending Whisper, OpenAI's Speech-to-Text Model","Abhishek Divekar, Yosub Jung, Roshni Tayal",Academic,2022-12-05,ut-mscs-case-studies-in-ml-project,whisper-wer-noisy-vs-quiet.png,https://adivekar-utexas.github.io/files/UTCS-M...,,We study the performance of OpenAI's Whisper m...,"Helped formulate research question, recorded a..."
1,Asking the Right Questions: Question Paraphras...,"Abhishek Divekar, Alex Stoken",Academic,2021-05-06,ut-mscs-nlp-project,asking-the-right-questions-table.png,https://adivekar-utexas.github.io/files/UTCS-N...,,A common issue when asking questions is that t...,"Formulated research question and GROK metric, ..."
2,Autonomous agents for realtime multiplayer ice...,"Abhishek Divekar, Jason Housman, Ankita Sinha,...",Academic,2020-12-14,ut-mscs-deep-learning-project,autonomous-agents-ice-hockey.png,https://adivekar-utexas.github.io/files/UTCS-D...,,We design an automated agent to play 2-on-2 ga...,"Generated training dataset of 40k images, code..."
3,SearchDistribute: webscraping search results o...,Abhishek Divekar,Academic,2017-09-08,vjti-search-distribute,,,https://github.com/ARDivekar/SearchDistribute,"retrieves upto 250,000 Search engine results p...",Sole contributor.


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [77]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [78]:
loc_dict = {}

for row, item in projects.iterrows():
    
    md_filename = str(item.project_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.project_date) + "-" + item.url_slug 
    year = item.project_date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: projects" + "\n"
    md += f'urlslug: "{item.url_slug }"\n'
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Project"\n'
    md += "permalink: /projects/" + html_filename + "\n"
    if len(str(item.contributors)) > 3:
        md += 'contributors: "' + item.contributors + '"\n'
    if len(str(item.my_contribution)) > 3:
        md += 'contribution: "' + item.my_contribution + '"\n'
    md += "date: " + str(item.project_date) + "\n"
    if len(str(item.teaser_url)) > 5:
        md += "teaserurl: '" + item.teaser_url + "'\n"
    report_url = None
    if len(str(item.report_url)) > 5:
        report_url = str(item.report_url)
        md += "reporturl: '" + item.report_url + "'\n"
    code_url = None
    if len(str(item.code_url)) > 5:
        code_url = str(item.code_url)
        md += "codeurl: '" + item.code_url + "'\n"
    if len(str(item.excerpt)) > 5:
        md += "excerpt: '" + html_escape(item.excerpt) + "'\n"
           
    md += "---\n"

    more_info = []
    if code_url is not None:
        more_info.append(f'[[Code]({code_url})]')
    if report_url is not None:
        more_info.append(f'[[Technical report]({report_url})]')
    if len(item.contributors) > 0:
        md += f"\n{item.contributors}\n"
    if len(str(item.excerpt)) > 4:
        md += f"\n**Description:**\n{html_escape(item.excerpt)}\n"
    if len(str(item.my_contribution)) > 4:
        md += f"\n**My contribution:**\n{html_escape(item.my_contribution)}\n"
    if len(more_info) > 0:
        md += f"\n**Resources:** {' '.join(more_info)}\n"
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [79]:
!ls ../_projects

2017-09-08-vjti-search-distribute.md
2020-12-14-ut-mscs-deep-learning-project.md
2021-05-06-ut-mscs-nlp-project.md
2022-12-05-ut-mscs-case-studies-in-ml-project.md


In [80]:
!cat ../_projects/2020-12-14-ut-mscs-deep-learning-project.md

---
title: "Autonomous agents for realtime multiplayer ice-hockey"
collection: projects
urlslug: "ut-mscs-deep-learning-project"
type: "Academic"
permalink: /projects/2020-12-14-ut-mscs-deep-learning-project
contributors: "Abhishek Divekar, Jason Housman, Ankita Sinha, Alex Stoken"
contribution: "Generated training dataset of 40k images, coded and trained multi-task CenterNet model for vision stage of pipeline, wrote sections of report."
date: 2020-12-14
teaserurl: 'autonomous-agents-ice-hockey.png'
reporturl: 'https://adivekar-utexas.github.io/files/UTCS-Deep-Learning-Final-Autonomous-agents-for-realtime-multiplayer-ice-hockey.pdf'
excerpt: 'We design an automated agent to play 2-on-2 games in SuperTuxKart IceHockey. Our two-stage system composes of a &quot;vision&quot; stage which takes as input the image of the player&apos;s Field of View and predicts world-state attributes. For vision, we train a multi-task CenterNet model (with U-Net backend), to predict whether hockey puck was on